In [2]:
import os
import nltk
import spacy
import es_core_news_sm
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from spacy.lang.es.stop_words import STOP_WORDS
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fonchopinto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/fonchopinto/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
# Inicializar el lematizador de spaCy
nlp = spacy.load('es_core_news_sm')

# Definir el directorio donde están los archivos .txt
directorio = 'DiscursosOriginales/'  # Reemplaza esto con tu directorio

# Lista para almacenar los contenidos de los archivos
contenidos = []

# Leer los textos y combinarlos en un solo corpus
for filename in os.listdir(directorio):
    if filename.endswith('.txt'):
        with open(os.path.join(directorio, filename), 'r', encoding='utf-8') as file:
            contenido = file.read()
            print(f'Archivo {filename} leído')

            # Tokenizar y lematizar el contenido
            tokens = word_tokenize(contenido.lower())# Convertir a minúsculas
            tokens_sin_stopwords = [word for word in tokens if word not in STOP_WORDS]
            doc = nlp(' '.join(tokens_sin_stopwords))
            lemas = [token.lemma_ for token in doc]

            # Convertir los lemas nuevamente a texto
            corpus_lematizado = ' '.join(lemas)
            contenidos.append(corpus_lematizado)


Archivo 91265.txt leído
Archivo 83107.txt leído
Archivo 79915.txt leído
Archivo 153144.txt leído
Archivo 94447.txt leído
Archivo 84898.txt leído
Archivo 149691.txt leído
Archivo 89628.txt leído
Archivo 85012.txt leído
Archivo 85006.txt leído
Archivo 80142.txt leído
Archivo 96720.txt leído
Archivo 75444.txt leído
Archivo 86296.txt leído
Archivo 91891.txt leído
Archivo 164307.txt leído
Archivo 100454.txt leído
Archivo 80552.txt leído
Archivo 136643.txt leído
Archivo 84907.txt leído
Archivo 102280.txt leído
Archivo 101951.txt leído
Archivo 71541.txt leído
Archivo 90595.txt leído
Archivo 76776.txt leído
Archivo 96872.txt leído
Archivo 96866.txt leído
Archivo 85952.txt leído
Archivo 164266.txt leído
Archivo 94863.txt leído
Archivo 73020.txt leído
Archivo 101415.txt leído
Archivo 80396.txt leído
Archivo 153186.txt leído
Archivo 75121.txt leído
Archivo 100509.txt leído
Archivo 136905.txt leído
Archivo 86042.txt leído
Archivo 163938.txt leído
Archivo 81705.txt leído
Archivo 102450.txt leído
Ar

In [146]:
print(contenidos[0])

: disfrutir sol invierno . gabriela mistral decir “ amar ciudad ” agregar belleza ciudad ennoblecer fealdad envilecer . importante ciudad hogar . conocer país cuidar ciudad , chile aprender . importante , ver cabl , cable necesario , empresa requerir cabl telecomunicación , telefonía fijo televisión instalar cable , cabl quedar desuso preocupo sacarlo . cable usado , estar ver lograr propósito : ciudad bonito , hermoso disfrutar él , ciudad seguro , cable desuso representar peligro . poner campaña “ chao cable ” , ¿ él acordar “ chao suegra ” , ¿ y reir ? suegrar . “ chao suegra ” pareja joven tener vivir padre suegro , tener oportunidad vivienda ir feliz , ir iniciar vida autonomía , independencia . “ chao cabl ” ciudad linda , bonitar disfrutar él , seguro . mes marzo año , logrado sacar 647 kilómetro cable desuso . distancia santiago temuco , distancia santiago vallenar . : meter llegar -ant terminar gobierno- sacar 2 mil kilómetro cable desuso . tarea . querer agradecer , lugar , t

In [4]:
model = Word2Vec(tokens, vector_size=100, window=5, min_count=1, sg=0)

In [5]:
vectores_por_texto = []
for texto_tokens in contenidos:
    vectores_por_token = [model.wv[token] for token in texto_tokens if token in model.wv]
    if vectores_por_token:
        # Promediar los vectores para obtener un vector representativo del texto
        vector_promedio = sum(vectores_por_token) / len(vectores_por_token)
        vectores_por_texto.append(vector_promedio)
    else:
        # Si no hay tokens en el modelo, agregar un vector de ceros
        vectores_por_texto.append([0] * model.vector_size)

In [7]:
type(vectores_por_texto)

list

In [23]:
consulta = str(input("Ingrese su consulta: "))
tokens_consulta = word_tokenize(consulta.lower())
tokens_sin_stopwords = [word for word in tokens_consulta if word not in STOP_WORDS]
doc = nlp(' '.join(tokens_sin_stopwords))
lemas = [token.lemma_ for token in doc]
consulta_lematizada = ' '.join(lemas)
embedding_consulta = [model.wv[token] for token in consulta_lematizada if token in model.wv]
vector_promedio = sum(embedding_consulta) / len(embedding_consulta)
embedding_consulta = vector_promedio.reshape(1, -1)


In [24]:
embedding_consulta

array([[-0.06090792,  0.08880129,  0.04257484,  0.1626564 , -0.03595069,
        -0.06671556,  0.10590803,  0.21727014, -0.20195113, -0.14700766,
         0.03764452, -0.10747821,  0.02942025,  0.08698068,  0.00203066,
        -0.08868967,  0.22386712,  0.06375731, -0.16005827, -0.3064683 ,
        -0.02341714, -0.00526198,  0.20234099, -0.0046511 , -0.00720756,
         0.00784368, -0.13740066,  0.16092029, -0.09740801,  0.12011885,
         0.15247516, -0.08910036,  0.04106846, -0.15164922, -0.10947861,
         0.09177454,  0.06741792, -0.04652753, -0.07505935, -0.03175658,
         0.07092072, -0.05414898, -0.13334809,  0.04442001,  0.04026859,
        -0.03890421, -0.09140781, -0.08516832,  0.04664858,  0.07031009,
         0.03800352, -0.16040148, -0.09979488, -0.02308897, -0.14441887,
        -0.0312277 ,  0.05646912, -0.08487309, -0.05095953,  0.02197694,
        -0.00067157, -0.05832019,  0.22054304, -0.01594426, -0.12530847,
         0.19723085,  0.03143917,  0.13885178, -0.2

In [25]:
similitudes = []
counter = 0
for vector in vectores_por_texto:
    vector = vector.reshape(1, -1)
    similitud = cosine_similarity(embedding_consulta, vector)[0][0]
    similitudes.append((contenidos[counter], similitud))
    counter += 1

# Ordenar por similitud descendente
similitudes = sorted(similitudes, key=lambda x: x[1], reverse=True)

In [26]:
for texto, similitud in similitudes[:5]:  # Muestra los 5 textos más relevantes
    print(f'Similitud: {similitud}')
    print(texto)
    print('-' * 30)

Similitud: 0.9999998211860657
tarde : historia enseño chile nacer encuentro mundo , encontrar posibilitado océano navegación . recoger forma elocuente poema escrito alonso ercilla , “ chile , fértil provincia , señalado . región antártico famoso , remoto nación respetado , fuerte , principal poderoso : gente producir granada , altivo , gallardar belicosa rey jamás regido , extranjero dominio sometido ” . resumir encuentro mundo . dar paso importante fortalecer proteger , resguardar eficacia vocación , destino acompaña chile nacimiento . reafirmar derecho , presencia soberanía continente blanco , continente paz , promulgar estatuto antártico . chile poderoso razón sostener derechos presencia continente antártico , carácter histórico tiempos colonia , carácter jurídico carácter geográfico . razón estrecho magallán , “ mar estaño ” llamar gabriela mistral , lugar momento reafirmar presencia continente antártico . puerta entrada continente antártico , puntar arena estrecho magallán , graci